In [15]:
%run ConfigFile.ipynb  ##import ConfigFile as cfg
%run DB_Connection.ipynb ##import DB_Connection as dbc
import pyodbc
import copy
import math
from datetime import datetime
import logging
import time
import os
import sys
from io import StringIO
import ast 

import pandas as pd
import numpy as np
import re

# Modelling and evaluation imports
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import chi2
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import pickle

In [7]:
#creating a unique id with the help of current date timestamp
st = datetime.fromtimestamp(time.time()).strftime('%d%m%Y%H%M%S')
clientname=client_name['clientname']
logging.basicConfig(filename=clientname+"_"+st+".log",level=logging.DEBUG,format="%(asctime)s:%(levelname)s:%(message)s")
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger(clientname)

<Logger SAAB (DEBUG)>

In [8]:
today_timestamp = datetime.now()

### Stratified Sampling

In [9]:
'''
    This function performs a stratified sampling on the input data frame 
    Input:
        input_dataframe : on which the sampling needs to be performed
    Output:
        Returns the train data, train labels, test data and test labels
'''
def stratified_sampling(input_dataframe):
    X = input_dataframe['Preprocessed Transcripts']
    y = input_dataframe['call_type']
    sss = StratifiedShuffleSplit(n_splits=10,test_size=0.2,random_state=32)
    sss.get_n_splits(X, y)
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    return  X_train, y_train, X_test, y_test

### Feature selection

In [10]:
'''
    This function performs a featre selection based on the chi2 scores to exclude less weightage score
    Input:
        X_train : train data
        y_train : train labels
        no_of_features : 
        threshold = chi2 square threshold value
    Output:
        Dataframe of the best features
'''
def feature_selection(X_train,y_train,no_of_features,chi_square_threshold):
    # Create dummies for classes
    y_train_dummies = pd.get_dummies(y_train)

    # Convert to token counts
    count_vec = CountVectorizer(ngram_range=(2,3),binary=True)
    X_train_count = count_vec.fit_transform(X_train)
    transformer = TfidfTransformer()
    X_train_transformed = transformer.fit_transform(X_train_count)
    
    columns = list(y_train_dummies.columns)
    features_final=[]
    chi_probability = []
    for col in columns:
        chi_score = chi2(X_train_transformed, y_train_dummies[col])[0]
        chi_prob = chi2(X_train_transformed, y_train_dummies[col])[1]
        features = count_vec.get_feature_names()
        chi_table = pd.DataFrame({'Features':features, 'Chisquare':chi_score , 'Chi_Square_Prob':chi_prob})
        chi_table_cutoff = chi_table.loc[(chi_table["Chisquare"] > chi_square_threshold)] 
        if len(chi_table_cutoff)>no_of_features:
            chi_table = chi_table_cutoff.sort_values(by='Chisquare', ascending=False).head(no_of_features)
        else:
            chi_table = chi_table_cutoff
        features_final.append(chi_table['Features'].tolist())
        chi_probability.append(chi_table['Chisquare'].tolist())
    features_all = [item for sublist in features_final for item in sublist]
    chi_values_all = [item for sublist in chi_probability for item in sublist]
    features_all_table = pd.DataFrame({'Features':features_all , 'Chisquare':chi_values_all})
    features_best_chi = pd.DataFrame(features_all_table.groupby(['Features'], as_index=False, sort=False)['Chisquare'].max())
    
    return  features_best_chi

### Find best k features

In [11]:
'''
    This function identifies the best numebr features to be used
    Input:
        features_best_chi : best features by chi2 score
        X_train : train data
        y_train : train labels
        X_test : test data
        y_test : test labels
        
    Output:
        Return the optimal number of features to consider
'''
def find_best_k_features(features_best_chi,X_train,y_train,X_test,y_test):
    k_val = features_best_chi.shape[0]
#     print((k_val))
    scores_f1 = []
    for k in range(1, int(k_val), 1):
        features_table = features_best_chi.sort_values('Chisquare', ascending=False).head(k)
        # Bi gram
        model = Pipeline([('vect', CountVectorizer(vocabulary=features_table.Features,ngram_range=(2,3))),
                         ('tfidf', TfidfTransformer()),
                         ('clf',  CalibratedClassifierCV(LinearSVC(penalty="l2", dual=False, tol=1e-3),cv=KFold(n_splits=3))),])#LinearSVC(penalty="l2", dual=False,tol=1e-3)
        # Training the Model
        model.fit(X_train, y_train)
        # Scoring the Model
        predicted = model.predict(X_test)
        scores_f1.append(f1_score(y_test, predicted, average='weighted'))

    kvals_list = list(range(1,int(k_val),1))

    scores_f1_table = pd.DataFrame({'K_values':kvals_list, 'F1-Score':scores_f1})
    kval_top = scores_f1_table.sort_values(by='F1-Score', ascending=False).head(n=10)
    kval_optimum = int(kval_top.iloc[0,0])
    
    return kval_optimum

### Training

In [12]:
'''
    This function trains the final model
    Input:
        features_table_chi_top : top best features selected
        X_train : train data
        y_train : train labels
        
    Output:
        Returns the trained model
'''
def train_model(features_table_chi_top,X_train,y_train):
    #Using calibrated classifier 
    model = Pipeline([('vect', CountVectorizer(vocabulary=features_table_chi_top.Features,ngram_range=(2,3))),
                         ('tfidf', TfidfTransformer()),
                         ('clf',  CalibratedClassifierCV(LinearSVC(penalty="l2", dual=False,tol=1e-3),cv=KFold(n_splits=3)))])

    # Training the Model
    trained_model = model.fit(X_train, y_train)
    return trained_model

### Write to DW

In [ ]:
'''
    Function to write the dataframe to DW
  
    Input :
        call_types_df : dataframe that consists of labels of the call types assigned for each call
            
'''
def write_df_to_dw(call_types_df):
    # Writing to DW
    today_timestamp = datetime.now()
    server = mysql['server']
    database = mysql['database']
    username = mysql['username']
    password = mysql['password']
    conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = conn.cursor()
    cursor.setinputsizes([(pyodbc.SQL_INTEGER,), (pyodbc.SQL_INTEGER,),(pyodbc.SQL_INTEGER,),(pyodbc.SQL_WVARCHAR,20,0),(pyodbc.SQL_WVARCHAR,0), (pyodbc.SQL_TYPE_TIMESTAMP), (pyodbc.SQL_WVARCHAR,50,0)])
    id_count = 0
    for index,row in word_count_df.iterrows():
        id_count += 1
        cursor.execute("INSERT INTO [dbo].[SAAB_ML_WORDCOUNT_FT]([RESULT_ID], [CALL_ID], [DOMAIN_ID], [TRANSCRIPT_TYPE], [TRANSCRIPT], [CREATED_DATE], [CREATED_BY]) values (?, ?, ?, ?, ?, ?, ?)", id_count, row['Call_ID'], row['Domain_ID'], row['Transcript_Type'], row['Transcript'],  today_timestamp, row['Created_By'])
    conn.commit()
    cursor.close()
    conn.close()

### Write model metrics to DW

In [3]:
'''
    Function to write the model metrics to DW
  
    Input :
        call_types_df : dataframe that consists of labels of the call types assigned for each call
            
'''
def write_metrics_df_to_dw(metrics_df):
    # Writing to DW
    today_timestamp = datetime.now()
    server = mysql['server']
    database = mysql['database']
    username = mysql['username']
    password = mysql['password']
    conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = conn.cursor()
#     cursor.setinputsizes([(pyodbc.SQL_INTEGER,), (pyodbc.SQL_INTEGER,),(pyodbc.SQL_INTEGER,),(pyodbc.SQL_WVARCHAR,20,0),(pyodbc.SQL_WVARCHAR,0), (pyodbc.SQL_TYPE_TIMESTAMP), (pyodbc.SQL_WVARCHAR,50,0)])
    id_count = 0
    for index,row in metrics_df.iterrows():
#         print(row)
#         master_id = fetch_data("select MAX(MASTER_ID) from [dbo].[SAAB_ML_MODEL_METRICS_FT]")
#         print(master_id)
#         if master_id is not None:
#             print("True")
#             id_count = fetch_data("select MAX(MASTER_ID) from [dbo].[SAAB_ML_MODEL_METRICS_FT]") + 1
#         else:
#             print("False")
        id_count = 1
        cursor.execute("INSERT INTO [dbo].[SAAB_ML_MODEL_METRICS_FT]([MASTER_ID], [MODEL_NAME], [METRIC_NAME], [VALIDATION_EVAL_METRIC], [CREATED_DATE], [CREATED_BY]) values (?, ?, ?, ?, ?, ?)", id_count, row['Model_Name'], row['Metric_Name'], row["Validation_Eval_Metric"], today_timestamp, row['Created_By'])
    conn.commit()
    cursor.close()
    conn.close()

### Main function

In [18]:
def main():
    logging.debug("Starting....")
    # Reading data from local system and azure data
    try:
        domain_id = 101
        created_by = "MDXC"
        top_level_container = "preprocessed-transcripts"

        manual_label_data = pd.read_excel("CallTypeLabels.xlsx",sheet_name="Sheet1")
        blob_file = fetch_data_from_blob(top_level_container)
        preprocessed_data = pd.read_csv(StringIO(blob_file))
        call_name_data = fetch_data("select call_ID,file_name from [dbo].[Fact_Audio_Processed];")

    except Exception as err:
            logging.error("Error occured while connecting to storage account :"+str(err.args)+"\nTraceback :"+str(err.with_traceback))
            sys.exit(1)
    try:
        # Merging data for input into training the model
        logging.debug("Merging data in progress....")
        merged_label_data = pd.merge(manual_label_data,call_name_data,left_on="CallName",right_on="file_name")
        merged_label_data = merged_label_data[['call_ID','call_type']]
        merged_label_data = pd.merge(merged_label_data,preprocessed_data,left_on="call_ID",right_on="Call_ID")
        merged_label_data = merged_label_data[['call_ID','call_type','Preprocessed Transcripts']]
        merged_label_data['Preprocessed Transcripts'] = merged_label_data['Preprocessed Transcripts'].apply(lambda x: " ".join(ast.literal_eval(x.replace("_"," "))))
        merged_label_data
        
        #Stratified Sampling
        logging.debug("Stratified sampling in progress....")
        X_train, y_train, X_test, y_test = stratified_sampling(merged_label_data)

        # Feature selection
        logging.debug("Finding best features in progress....")
        features_best_chi = feature_selection(X_train,y_train,1000,0)

        # Finding best K-features
        kval_optimum = find_best_k_features(features_best_chi,X_train, y_train, X_test, y_test)

        #Training data with optimal features
        logging.debug("Training in progress....")
        features_table_chi_top = features_best_chi.sort_values('Chisquare', ascending=False).head(kval_optimum)
        final_trained_model = train_model(features_table_chi_top,X_train, y_train)
        
        # Save model to disk
        filename = 'finalized_call_type_model.sav'
        pickle.dump(final_trained_model, open(filename, 'wb'))
        
        # Predict on test set
        logging.debug("Prediction and scoring in progress....")
        predicted_labels = final_trained_model.predict(X_test)
        prec_recall_fscore_support = precision_recall_fscore_support(y_test, predicted_labels)
        accuracy = accuracy_score(y_test,predicted_labels)
        print(accuracy)
    
        
        # Write model metrics to table
        logging.debug("Writing model metrics to DW in progress...")
#         metric_df = pd.DataFrame([["CallTypes_Classification_M1","Accuracy",accuracy,"MDXC"]], columns=['Model_Name','Metric_Name',"Validation_Eval_Metric","Created_By"])
#         write_metrics_df_to_dw(metric_df) 
    
    except Exception as err:     
        logging.error("Error occured : "+str(err.args)+"\nTraceback :"+str(err.with_traceback))
        sys.exit(1)
    finally:
        logging.debug("End of call types classification")

In [19]:
if __name__ == "__main__":
    main()

DEBUG - Starting....
DEBUG - Starting new HTTPS connection (1): saabstorageresource.blob.core.windows.net:443
DEBUG - https://saabstorageresource.blob.core.windows.net:443 "GET /preprocessed-transcripts?restype=container&comp=list HTTP/1.1" 200 None
DEBUG - https://saabstorageresource.blob.core.windows.net:443 "GET /preprocessed-transcripts/Preprocessed_Transcripts_29012020161128.csv HTTP/1.1" 206 170282
DEBUG - Merging data in progress....
DEBUG - Stratified sampling in progress....
DEBUG - Finding best features in progress....
DEBUG - Training in progress....
DEBUG - Prediction and scoring in progress....
DEBUG - Writing model metrics to DW in progress...
DEBUG - End of call types classification


0.6875


### Predict on unseen data

### Write the labelled data to DW

In [25]:

# Reading data from local system and azure data
domain_id = 101
created_by = "MDXC"
top_level_container = "preprocessed-transcripts"

manual_label_data = pd.read_excel("CallTypeLabels.xlsx",sheet_name="Sheet1")
blob_file = fetch_data_from_blob(top_level_container)
preprocessed_data = pd.read_csv(StringIO(blob_file))
call_name_data = fetch_data("select call_ID,file_name from [dbo].[Fact_Audio_Processed];")

merged_label_data = pd.merge(manual_label_data,call_name_data,left_on="CallName",right_on="file_name")
merged_label_data = merged_label_data[['call_ID','call_type']]
merged_label_data = pd.merge(merged_label_data,preprocessed_data,left_on="call_ID",right_on="Call_ID")
merged_label_data = merged_label_data[['call_ID','call_type','Preprocessed Transcripts']]
merged_label_data['Preprocessed Transcripts'] = merged_label_data['Preprocessed Transcripts'].apply(lambda x: " ".join(ast.literal_eval(x.replace("_"," "))))
merged_label_data['Call_Type_ID'] = np.where(merged_label_data['call_type']=="Enquiry/Support",1,2)
merged_label_data

DEBUG - Starting new HTTPS connection (1): saabstorageresource.blob.core.windows.net:443
DEBUG - https://saabstorageresource.blob.core.windows.net:443 "GET /preprocessed-transcripts?restype=container&comp=list HTTP/1.1" 200 None
DEBUG - https://saabstorageresource.blob.core.windows.net:443 "GET /preprocessed-transcripts/Preprocessed_Transcripts_29012020161128.csv HTTP/1.1" 206 170282


,call_ID,call_type,Preprocessed Transcripts,Call_Type_ID
0,136043333,Complaint,hello good help want term insurance policy ben...,2
1,1444129832,Enquiry/Support,hello help sorry want convert term life policy...,1
2,288021220,Enquiry/Support,hello good evening help actually premium month...,1
3,800562374,Enquiry/Support,hello good help height dumb policy year expire...,1
4,2146526476,Complaint,welcome life help high angel policy number que...,2
...,...,...,...,...
72,61876574,Enquiry/Support,good customer care help today want unit link p...,1
73,1344343262,Complaint,life help today face problem policy unit link ...,2
74,1791382475,Complaint,life customer care help today actually policy ...,2
75,1692229584,Complaint,good life customer care help today actually pr...,2


In [28]:
server = 'saab-server-resource.database.windows.net'
database = 'SAAB_DW_Resource'
username = 'saabadmin'
password = 'p@$$w0rd'
conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()
id_count = 0
for index,row in merged_label_data.iterrows():
    id_count += 1
    cursor.execute("INSERT INTO [dbo].[SAAB_ML_CALLTYPES_FT]([MASTER_ID],[DOMAIN_ID],[CALLTYPE_ID],[CALL_ID],[CREATED_DATE],[CREATED_BY]) values (?, ?, ?, ?, ?, ?)", id_count, 101,row['Call_Type_ID'], row['call_ID'], today_timestamp,'MDXC') 
conn.commit()
cursor.close()
conn.close()

### Writing to call types master table

In [5]:
# def write_call_types_dm_table():
master_df = pd.DataFrame()
master_df['master_id'] = [1,2]
# master_df['domain_id'] = [101,101]
master_df['calltype_id'] = [1,2]
master_df['calltype'] = ['Enquiry/Support','Complaint']
#     master_df['keywords'] = ['{"Section_1":["good morning", "good afternoon"],"Section_2":["welcome to", "you are through to","thank you for calling"],"Section_3":["how can i help", "how may i help", "what can I assist", "what may i assist"]}','{"Section_1":["thanks for calling","thank you for calling","have a nice day","have a good day","thank you for contacting", "have a great day", "is there anything else"]}']
# master_df['created_date'] = [today_timestamp,today_timestamp]
# master_df['created_by'] = ["MDXC","MDXC"]
master_df

,master_id,calltype_id,calltype
0,1,1,Enquiry/Support
1,2,2,Complaint


In [7]:
server = 'saab-server-resource.database.windows.net'
database = 'SAAB_DW_Resource'
username = 'saabadmin'
password = 'p@$$w0rd'
conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()
for index,row in master_df.iterrows():
    cursor.execute("INSERT INTO [dbo].[SAAB_ML_MASTER_CALLTYPES_DM]([RESULT_ID],[DOMAIN_ID],[CALLTYPE_ID],[CALLTYPE],[CREATED_DATE],[CREATED_BY]) values (?, ?, ?, ?, ?, ?)", row['master_id'], 101,row['calltype_id'], row['calltype'], today_timestamp,'MDXC') 
conn.commit()
cursor.close()
conn.close()